# Calibración y análisis — Modelo de Tanques

In [ ]:

import pandas as pd
import numpy as np
from tank_model import TankModel, ModelConfig, nse, kge, bias_pct
from tank_model.parameters import Parameters
from tank_model.io import load_csv, subset_period, tag_hydrology
from tank_model.et import ensure_pet

# 1) Cargar datos
df = load_csv('../data/example_forcing.csv')
df.head()


In [ ]:

# 2) Asegurar PET con Cenicafé (si tienes Tmean_C y Rs_MJ_m2_d en tu CSV)
# df = ensure_pet(df, method='cenicafe', a=0.0135, b=17.78)
# Para el ejemplo ya existe PET_mm:
df = ensure_pet(df, method='column')
df[['P_mm','PET_mm']].plot(title='P y PET', figsize=(9,3))


In [ ]:

# 3) Configurar y correr
cfg = ModelConfig(dt_hours=24.0, area_km2=5.0, route=True)
p = Parameters()
m = TankModel(p, cfg)
sim = m.run(df)
sim[['Qout_mm']].plot(title='Qout (mm/dt)', figsize=(9,3))


In [ ]:

# 4) Análisis seco/húmedo y métricas por régimen
reg = tag_hydrology(df, col='P_mm', dry_q=0.25, wet_q=0.75)
sim['regime'] = reg
sim.groupby('regime')['Qout_mm'].describe()


In [ ]:

# 5) NSE/KGE por subperíodos (ejemplo sintético con obs = sim + ruido)
obs = sim['Qout_mm'] * (1 + np.random.normal(0, 0.1, size=len(sim)))
print('NSE total:', nse(obs, sim['Qout_mm']))
for r in ['dry','normal','wet']:
    mask = sim['regime'] == r
    print(r, 'NSE=', nse(obs[mask], sim.loc[mask,'Qout_mm']))
